In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from matplotlib import pyplot as plt
import pickle
import sys
sys.path.append('./../')
sys.path.append('./../../')
import Portfolio_value as pv
import portfolio_insurance as pi
import scipy

In [2]:
df =  pd.read_csv('./data/snp2.csv')

In [3]:
rf = df['RF'].mean() * 12

In [4]:
p_val = np.load('./result/normal/p_val_0.05.npy', allow_pickle=True)
w_val = np.load('./result/normal/w_val_0.05.npy', allow_pickle=True)

p_val = p_val.reshape(1)
p_val = p_val[0]

w_val = w_val.reshape(1)
w_val = w_val[0]

In [5]:
bear_year = np.load('./result/year/normal_year.npy', allow_pickle=True).reshape(1)[0]


In [6]:
upper_years = [35]

In [7]:


# model_names = ['BH_80', 'BH_75', 'BH_70', 'BH_65', 'BH_60', 'BH_55', 'BH_50', 'BH_45', 'BH_40', 'BH_35', 'BH_30', 'SL_100', 'SP_100', 'CPPI_100_70', 'TIPP_100_70'] # main table
model_names = ['BH_75', 'BH_70', 'BH_65', 'BH_60', 'BH_55', 'BH_50', 'SL_100', 'SP_100', 'CPPI_100_70', 'TIPP_100_70'] # main table




In [8]:
bear_perform = {}
for model_name in model_names:
    bear_perform[model_name] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}

In [9]:
# PI_Korea랑은 코드가 좀 다름

def get_metric_for_PI(retire_year, k, upper_year, model_name, withrate, const_or_dyna, df, init_wealth, p_val, w_val):
    # bequest
    p_amounts = p_val['{}_rate{}_{}'.format(model_name, withrate, const_or_dyna)][retire_year]
    bequest = p_amounts[12*upper_year]*init_wealth*(1-withrate)/100
    
    # withdraw_amount
    w_amounts = w_val['{}_rate{}_{}'.format(model_name, withrate, const_or_dyna)][retire_year]
    withdraw_amount = sum(w_amounts)
    
    # depletion time
    if np.argwhere(p_amounts == 0).size > 0 : # 고갈이 되면
        dep_time = np.argwhere(p_amounts == 0)[0][0]
    elif np.argwhere(p_amounts == 0).size == 0: # 고갈이 안되면
        dep_time = upper_year*12
        
    # max decline
    dec = np.min(p_amounts)/p_amounts[0]
    dec = dec - 1
    
    # underwater duration
    start_date = datetime(retire_year, 1, 1)
    end_date = start_date + relativedelta(years=upper_year)
    
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m')
    df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    
    under_count = 0
    for i in range(upper_year*12):
        if p_amounts[i]*df.iloc[i]['CPI'] < p_amounts[0]*df.iloc[0]['CPI'] :
             under_count+= 1
        
    return bequest / init_wealth * 100, withdraw_amount / init_wealth * 100, dep_time, dec * 100, under_count

In [10]:
init_wealth = 500000
withdraw_rate = 0.05
k=0
const_or_dyna = 'const'
upper_year = 35

for model_name in model_names:
    print(model_name)
    for years in bear_year[upper_year]:
        for year in years:
            res = get_metric_for_PI(year, k, upper_year, model_name, withdraw_rate, const_or_dyna, df, init_wealth, p_val, w_val)
            if res[2] < upper_year*12 : #고갈 시점이 upper_year보다 작으면 (이미 고갈이 되면)
                bear_perform[model_name]['dep'].append(100)
            elif res[2] >= upper_year*12: # 고갈이 안되면
                bear_perform[model_name]['dep'].append(0)
            bear_perform[model_name]['max'].append(res[3])
            bear_perform[model_name]['under'].append(res[4])
            bear_perform[model_name]['inc'].append(res[1])
            bear_perform[model_name]['beq'].append(res[0])
            bear_perform[model_name]['total'].append(res[1] + res[0])


BH_75
BH_70
BH_65
BH_60
BH_55
BH_50
SL_100
SP_100
CPPI_100_70
TIPP_100_70


In [11]:
avg, std, skew, kurto, SR = {}, {}, {}, {}, {}
ASR, MDD, AvDD, VaR, ES, semi, Omega, Omega2, Omega3 = {}, {}, {}, {}, {}, {}, {}, {}, {}
E, res, ret_sim, ret_sim_y = {}, {}, {}, {}

log_u = {}
quad_u_01 = {}
quad_u_05 = {}
quad_u_10 = {}
power_u_01 = {}
power_u_05 = {}
power_u_09 = {}
exp_u_01 = {}
exp_u_05 = {}
exp_u_10 = {}
exp_u_50 = {}


MPV_1 = {}
MPV_225 = {}
CPV = {}


for items in list(p_val.keys()):
    item = items.split('_')[0] + '_' + items.split('_')[1]
    ret_sim[item], ret_sim_y[item] = [], []
    avg[item], std[item] = [], []
    skew[item], kurto[item] = [], []
    MDD[item], AvDD[item] = [], []
    SR[item] = []
    VaR[item], ES[item], semi[item] = [], [], []
    Omega[item], Omega2[item], Omega3[item] = [], [], []

In [12]:
res_final = {}
for items in list(p_val.keys()):
    item = items.split('_')[0] + '_' + items.split('_')[1]
    res_final[item] = [pd.Series(i[1]) for i in list(p_val[items].items())]


In [13]:

for k in range(len(bear_year[35][0])):    
    for items in list(p_val.keys()):
        item = items.split('_')[0] + '_' + items.split('_')[1]
        res[item] = res_final[item][k]
        ret_m = (res[item] - res[item].shift(1)) / (res[item].shift(1)+0.0000000000000001)
        ret_m = ret_m.iloc[1:]
#         ret_sim[item].extend(list(ret_m*12))
        
        avg_= np.mean(ret_m) * 12
        std_ = np.std(ret_m) * np.sqrt(12)
        avg[item].append(avg_)
        std[item].append(std_)
        skew_ = scipy.stats.skew(ret_m)
        kurto_ = scipy.stats.kurtosis(ret_m)
        skew[item].append(skew_)
        kurto[item].append(kurto_)
        
        SR_ = (avg_ - rf) / (std_+0.0000000000000001)
        SR[item].append(SR_)
        
        
        MDD[item].append(pi.MDD(res[item]))
        AvDD[item].append(pi.AvDD(res[item]))
        
        VaR[item].append(pi.VaR(ret_m, level=5, freq='Y'))
        ES[item].append(pi.ES(ret_m, level=5, freq='Y'))
        semi[item].append(pi.semi(ret_m, freq='Y'))
        
        Omega[item].append(pi.Omega(ret_m, threshold=0, freq='Y'))
        Omega2[item].append(pi.Omega(ret_m, threshold=-0.05, freq='Y'))
        

        
#         ret_sim_y[item].append((res[item].iloc[-1] - res[item].iloc[0])/res[item].iloc[0])
        


In [14]:
for items in list(p_val.keys()):
    item = items.split('_')[0] + '_' + items.split('_')[1]
    ret_sim[item] = pd.Series(ret_sim[item])
    ret_sim_y[item] = pd.Series(ret_sim_y[item])
    
    # 매달 계산한걸 평균
    avg[item] = np.array(avg[item])
    std[item] = np.array(std[item])
    
    skew[item] = np.array(skew[item])#ret_sim[item].skew()
    kurto[item] = np.array(kurto[item])#ret_sim[item].kurtosis()
    SR[item] = np.array(SR[item])#(avg[item].mean() - rf) / (std[item].mean() + 0.0000000000000001)
    
    MDD[item] = np.array(MDD[item])
    AvDD[item] = np.array(AvDD[item])
    
    VaR[item] = np.array(VaR[item])#pi.VaR(ret_sim_y[item], level=5, freq='Y')
    ES[item] = np.array(ES[item])#pi.ES(ret_sim_y[item], level=5, freq='Y')
    semi[item] = np.array(semi[item])#pi.semi(ret_sim_y[item], freq='Y')
    
    Omega[item] = np.array(Omega[item])#pi.Omega(ret_sim[item], threshold=0, freq='Y')
    Omega2[item] = np.array(Omega2[item])#pi.Omega(ret_sim[item], threshold=0, freq='Y')
    



In [15]:
for items in list(bear_perform.keys()):
    item = items.split('_')[0] + '_' + items.split('_')[1]
    
    bear_perform[items]['avg'] = avg[item]
    bear_perform[items]['std'] = std[item]
    bear_perform[items]['skew'] = skew[item]
    bear_perform[items]['kurto'] = kurto[item]
    bear_perform[items]['SR'] = SR[item]
    bear_perform[items]['MDD'] = MDD[item]
    bear_perform[items]['AvDD'] = AvDD[item]
    bear_perform[items]['VaR'] = VaR[item]
    bear_perform[items]['ES'] = ES[item]
    bear_perform[items]['semi'] = semi[item]
    bear_perform[items]['Omega'] = Omega[item]
    bear_perform[items]['Omega2'] = Omega2[item]
    
    

In [16]:
# Define all dictionaries and indices we want to calculate mean for
dicts = [bear_perform]
indices = model_names

# Use list comprehension to compute all means at once
# Store results in lists
a = [np.round(np.mean(d[i]['dep']),1) for i in indices for d in dicts]
b = [np.round(np.mean(d[i]['max']),1) for i in indices for d in dicts]
c = [np.round(np.mean(d[i]['under']),1) for i in indices for d in dicts]
d = [np.round(np.mean(d[i]['inc']),1) for i in indices for d in dicts]
e = [np.round(np.mean(d[i]['beq']),1) for i in indices for d in dicts]
f = [np.round(np.mean(d[i]['total']),1) for i in indices for d in dicts]

d1 = [np.round(np.mean(d[i]['avg'])*100,2) for i in indices for d in dicts]
d2 = [np.round(np.mean(d[i]['std'])*100,1) for i in indices for d in dicts]
d3 = [np.round(np.mean(d[i]['skew']),2) for i in indices for d in dicts]
d4 = [np.round(np.mean(d[i]['SR']),2) for i in indices for d in dicts]
d5 = [np.round(np.mean(d[i]['MDD'])*100,1) for i in indices for d in dicts]
d6 = [np.round(np.mean(d[i]['AvDD'])*100,1) for i in indices for d in dicts]
d7 = [np.round(np.mean(d[i]['VaR'])*100,1) for i in indices for d in dicts]
d8 = [np.round(np.mean(d[i]['ES'])*100,1) for i in indices for d in dicts]
d9 = [np.round(np.mean(d[i]['semi'])*100,1) for i in indices for d in dicts]
d10 = [np.round(np.mean(d[i]['Omega']),2) for i in indices for d in dicts]
d11 = [np.round(np.mean(d[i]['Omega2']),1) for i in indices for d in dicts]




# If you need to access the results separately, you can do so by indexing the lists:
# For example, a1 corresponds to a_results[0], a2 corresponds to a_results[1], etc.

In [17]:
def format_values(lst, highest_best=True):
    if highest_best:
        best_val = max(lst)
    else:
        best_val = min(lst)

    sorted_lst = sorted(lst, reverse=highest_best)
    second_best_val = sorted_lst[1] if len(sorted_lst) > 1 else None

    formatted_lst = []
    for val in lst:
        if val == best_val:
            formatted_lst.append(r"\textbf{" + str(val) + "}")
        elif val == second_best_val:
            formatted_lst.append(r"\underline{" + str(val) + "}")
        else:
            formatted_lst.append(str(val))

    return formatted_lst

# ... (rest of your code remains unchanged)

formatted_a = format_values(a, highest_best=False)
formatted_b = format_values(b)
formatted_c = format_values(c, highest_best=False)
formatted_f = format_values(f)
formatted_d10 = format_values(d10)
formatted_d11 = format_values(d11)

# The print statements remain the same.

n_bnh = 6
print('Depletion Prob. (\%) & & ' + ' & '.join(['{}'] * n_bnh).format(*formatted_a[:n_bnh]) + ' && ' + ' & '.join(['{}'] * (len(formatted_a) - n_bnh)).format(*formatted_a[n_bnh:]) + '\\\\')
print('Max. decline (\%) & & ' + ' & '.join(['{}'] * n_bnh).format(*formatted_b[:n_bnh]) + ' && ' + ' & '.join(['{}'] * (len(formatted_b) - n_bnh)).format(*formatted_b[n_bnh:])+ '\\\\')
print('Underwater duration (month) & & ' + ' & '.join(['{}'] * n_bnh).format(*formatted_c[:n_bnh]) + ' && ' + ' & '.join(['{}'] * (len(formatted_c) - n_bnh)).format(*formatted_c[n_bnh:])+ '\\\\')
print('Total withdrawal and Bequest (\%) & & ' + ' & '.join(['{}'] * n_bnh).format(*formatted_f[:n_bnh]) + ' && ' + ' & '.join(['{}'] * (len(formatted_f) - n_bnh)).format(*formatted_f[n_bnh:])+ '\\\\')

print('Omega ratio (0\%) & & ' + ' & '.join(['{}'] * n_bnh).format(*formatted_d10[:n_bnh]) + ' && ' + ' & '.join(['{}'] * (len(formatted_d10) - n_bnh)).format(*formatted_d10[n_bnh:])+ '\\\\')
print('Omega ratio (-5\%) & & ' + ' & '.join(['{}'] * n_bnh).format(*formatted_d11[:n_bnh]) + ' && ' + ' & '.join(['{}'] * (len(formatted_d11) - n_bnh)).format(*formatted_d11[n_bnh:])+ '\\\\')


Depletion Prob. (\%) & & 11.8 & 11.8 & 11.8 & 11.8 & 11.8 & 20.6 && 50.0 & \textbf{5.9} & \textbf{5.9} & 88.2\\
Max. decline (\%) & & -30.5 & -31.8 & -33.7 & -37.6 & -43.7 & -51.3 && -68.9 & \textbf{-26.3} & \underline{-29.4} & -99.4\\
Underwater duration (month) & & \underline{44.4} & 46.0 & 48.9 & 56.5 & 77.2 & 91.1 && 199.3 & \textbf{23.3} & 45.8 & 348.4\\
Total withdrawal and Bequest (\%) & & \underline{361.9} & 325.7 & 293.3 & 264.4 & 238.8 & 216.1 && 172.5 & 351.5 & \textbf{363.6} & 108.8\\
Omega ratio (0\%) & & 1.14 & 1.1 & 1.06 & 1.01 & 0.95 & 0.87 && 0.68 & \underline{1.17} & \textbf{1.2} & 0.13\\
Omega ratio (-5\%) & & 36.1 & 41.9 & 48.6 & 56.0 & 61.6 & \underline{67.4} && 24.4 & 47.9 & \textbf{67.7} & 7.3\\
